In [5]:
import numpy as np
import torch
import dnnlib
import legacy

# Load pre-trained model
network_pkl = 'ffhq.pkl'
device = torch.device('cuda')
with dnnlib.util.open_url(network_pkl) as f:
    G = legacy.load_network_pkl(f)['G_ema'].to(device)  # type: ignore

# Function to generate latent vectors and corresponding images
def generate_latents_and_images(num_samples, model):
    latents = []
    images = []
    for _ in range(num_samples):
        z = np.random.randn(1, model.z_dim)
        z_tensor = torch.from_numpy(z).to(device)
        img = model(z_tensor, None, truncation_psi=0.7, noise_mode='const')
        latents.append(z)
        images.append(img.cpu().detach().numpy())  # Move image tensor to CPU before converting to numpy
    return np.array(latents), np.array(images)

# Generate latent vectors for smiling and non-smiling images
num_samples = 100  # Number of samples for each class
latents_smiling, images_smiling = generate_latents_and_images(num_samples, G)
latents_not_smiling, images_not_smiling = generate_latents_and_images(num_samples, G)


In [6]:
from sklearn.linear_model import LinearRegression

# Create labels
labels_smiling = np.ones(num_samples)
labels_not_smiling = np.zeros(num_samples)

# Combine data
latents = np.vstack([latents_smiling, latents_not_smiling])
labels = np.hstack([labels_smiling, labels_not_smiling])

# Flatten latent vectors for linear regression
latents_flat = latents.reshape(latents.shape[0], -1)

# Fit linear regression model
regressor = LinearRegression()
regressor.fit(latents_flat, labels)

# Extract direction vector
direction_vector = regressor.coef_.reshape(latents.shape[1:])
np.save('smile_direction.npy', direction_vector)


In [7]:
import numpy as np
import torch
import dnnlib
import legacy
import PIL.Image
from sklearn.linear_model import LinearRegression

# Define the function to convert tensor to image
def tensor_to_image(tensor):
    tensor = tensor.squeeze(0).detach().cpu()  # Move tensor to CPU
    tensor = (tensor + 1.0) * 127.5
    tensor = tensor.clamp(0, 255)
    tensor = tensor.to(torch.uint8)
    tensor = tensor.permute(1, 2, 0)
    return PIL.Image.fromarray(tensor.numpy())

# Load pre-trained model
network_pkl = 'ffhq.pkl'
device = torch.device('cuda')
with dnnlib.util.open_url(network_pkl) as f:
    G = legacy.load_network_pkl(f)['G_ema'].to(device)  # type: ignore

# Load pre-trained model
# Assume G is already loaded as in the previous examples

# Function to generate latent vectors and corresponding images
def generate_latents_and_images(num_samples, model):
    latents = []
    images = []
    for _ in range(num_samples):
        z = np.random.randn(1, model.z_dim)
        z_tensor = torch.from_numpy(z).to(device)
        img = model(z_tensor, None, truncation_psi=0.7, noise_mode='const')
        latents.append(z)
        images.append(img.cpu().detach().numpy())  # Move image tensor to CPU before converting to numpy
    return np.array(latents), np.array(images)

# Generate latent vectors for samples
num_samples = 100  # Number of samples for each class
latents, images = generate_latents_and_images(num_samples, G)

from sklearn.linear_model import LinearRegression

# Fit linear regression model for age
regressor_age = LinearRegression()
regressor_age.fit(latents_flat, age_labels)

# Extract direction vector for age
age_direction = regressor_age.coef_.reshape(latents.shape[1:])
np.save('age_direction.npy', age_direction)

# Fit linear regression model for gender
regressor_gender = LinearRegression()
regressor_gender.fit(latents_flat, gender_labels)

# Extract direction vector for gender
gender_direction = regressor_gender.coef_.reshape(latents.shape[1:])
np.save('gender_direction.npy', gender_direction)


# Manipulate latent vectors to control age and gender
z_target = np.random.randn(1, G.z_dim)  # Random latent vector for demonstration
z_target += age_intensity * age_direction  # Manipulate age
z_target += gender_intensity * gender_direction  # Manipulate gender

# Generate new image
z_target_tensor = torch.from_numpy(z_target).to(device)
img_target = G(z_target_tensor, None, truncation_psi=0.7, noise_mode='const')

# Convert and save the manipulated image
img_target_pil = tensor_to_image(img_target)
img_target_pil.save('target_image.png')
